Author: [Thamme Gowda](http://www-scf.usc.edu/~tnarayan)

# Data Exploration


In [1]:
import os
import sys
import re
from collections import OrderedDict
from collections import Counter
import numpy as np

In [2]:
def read_input(dir_path, ext=".txt"):
    for f_path in filter(lambda x: x.endswith(ext), os.listdir(dir_path)):
        with open(os.path.join(dir_path, f_path)) as ptr:
            yield int(f_path.replace(ext, '')), ptr.read().strip()

data = [x for x in read_input("data")]
data = sorted(data)

In [3]:
def tokenize(txt, clean=True, lower=False):
    if clean:
        # replace all non alphabets with spaces
        txt = re.sub('[^a-zA-Z]+', ' ', txt)
    if lower:
        txt = txt.lower()
    return txt.split()

unigrams = {}
bigrams = {}
uni_counts = Counter()
bi_counts = Counter()
for id, txt in data:
    tokens = tokenize(txt, lower=True)
    bi_tokens = list(zip(tokens, tokens[1:]))
    unigrams[id] = tokens
    bigrams[id] =  bi_tokens
    uni_counts.update(tokens)
    bi_counts.update(bi_tokens)

In [4]:
uni_counts.most_common(50)

[('the', 37999),
 ('of', 17400),
 ('and', 12047),
 ('in', 10617),
 ('a', 10510),
 ('to', 10308),
 ('is', 7548),
 ('for', 4959),
 ('with', 4122),
 ('as', 3588),
 ('that', 3517),
 ('by', 3339),
 ('are', 3227),
 ('at', 2988),
 ('be', 2954),
 ('this', 2713),
 ('on', 2689),
 ('from', 2394),
 ('can', 1936),
 ('was', 1734),
 ('an', 1719),
 ('which', 1630),
 ('it', 1454),
 ('we', 1411),
 ('energy', 1374),
 ('fig', 1291),
 ('have', 1237),
 ('or', 1213),
 ('s', 1172),
 ('has', 1148),
 ('high', 1136),
 ('using', 1100),
 ('these', 1090),
 ('were', 1076),
 ('used', 1073),
 ('cells', 1061),
 ('c', 1049),
 ('m', 1028),
 ('been', 1026),
 ('e', 973),
 ('cell', 969),
 ('also', 966),
 ('d', 953),
 ('temperature', 926),
 ('two', 925),
 ('between', 923),
 ('not', 899),
 ('than', 884),
 ('i', 861),
 ('such', 836)]

In [5]:
bi_counts.most_common(50)

[(('of', 'the'), 5526),
 (('in', 'the'), 2911),
 (('to', 'the'), 2051),
 (('on', 'the'), 1243),
 (('and', 'the'), 1232),
 (('can', 'be'), 1164),
 (('for', 'the'), 1023),
 (('from', 'the'), 988),
 (('that', 'the'), 875),
 (('with', 'the'), 863),
 (('at', 'the'), 812),
 (('is', 'the'), 730),
 (('in', 'fig'), 729),
 (('by', 'the'), 724),
 (('of', 'a'), 718),
 (('in', 'a'), 713),
 (('due', 'to'), 707),
 (('with', 'a'), 632),
 (('to', 'be'), 608),
 (('it', 'is'), 606),
 (('as', 'a'), 560),
 (('has', 'been'), 510),
 (('shown', 'in'), 510),
 (('as', 'the'), 498),
 (('in', 'this'), 471),
 (('such', 'as'), 463),
 (('et', 'al'), 458),
 (('to', 'a'), 416),
 (('between', 'the'), 410),
 (('is', 'a'), 385),
 (('have', 'been'), 376),
 (('the', 'same'), 339),
 (('used', 'to'), 338),
 (('the', 'sample'), 334),
 (('and', 'a'), 322),
 (('for', 'a'), 318),
 (('which', 'is'), 317),
 (('based', 'on'), 312),
 (('by', 'a'), 292),
 (('i', 'e'), 288),
 (('number', 'of'), 275),
 (('the', 'first'), 266),
 (('a', 

In [6]:
ids = [x[0] for x in data]
N = len(ids)
# check that ids are sorted
assert not list(filter(lambda p: p[0] > p[1],  zip(ids, ids[1:])))
print("Number of docs = ", N)

Number of docs =  100


# Task 1 : Find common words between documents

For this task, you need to generate a matrix, where each entry contains the number of unique
common tokens (words) between each pair of documents. The output should include no headers for
rows or columns. The matrix should be symmetric, and follow the numbering conventions of the files.

The diagonal entries would be the count of unique terms in each document.
For example, the first number on the first line is the number of unique terms in 001.txt, the second
number on the second line is the number of unique terms in 002.txt, etc.
Similarly, the first element on the second line would be the number of unique terms that appear in
both 001.txt and 002.txt, the 23rd number on the 16th line is the number of unique common terms
between 023.txt and 016.txt, etc.

In [7]:
table = np.zeros((N, N), dtype=int)
uni_sets = {}

for id in unigrams:
    uni_sets[id] = set(unigrams[id])

for i in ids:
    for j in ids:
        if i == j:
            # size of the set
            table[i-1][i-1] = len(uni_sets[i])
        else:
            # intersection of two sets and size of the resulting set
            table[i-1][j-1] = len(uni_sets[i] & uni_sets[j])
            
## assert that matrix is symmteric
for i in range(N):
    for j in range(N):
        assert table[i][j] == table[j][i]
print("Matrix is symmetric")

Matrix is symmetric


In [17]:
OUT_FILE = 'task1-out.txt'
np.savetxt(OUT_FILE, table, fmt="%d", delimiter=' ') 
#!cat {OUT_FILE} | pr -w 80
#!fold -w 80 -s {OUT_FILE}
print(table)

[[1185  404  287 ...,  252  384  252]
 [ 404 1361  279 ...,  252  431  256]
 [ 287  279  821 ...,  351  302  246]
 ..., 
 [ 252  252  351 ...,  694  278  218]
 [ 384  431  302 ...,  278 1234  276]
 [ 252  256  246 ...,  218  276  600]]


# Task 2: Identifying words by frequency

A bigram is sequence of two consecutive tokens. The previous sentence, for example, contains the bigrams: (a bigram), (bigram is), (is sequence), (sequence of), (of two), (two consecutive), and (consecutive tokens).
Across the entire corpus find (1) the top 50 most frequent unigrams (single tokens), and (2) the top 50 most frequent bigrams.

The output should be a list of 100 lines, where the first 50 lines contain a single term each, in order of frequency, followed by 50 lines containing two terms each, in order of the bigram frequency.

In [10]:
MAX = 50
OUT_FILE = 'task2-out.txt'
lines = []
for term, count in uni_counts.most_common(MAX):
    lines.append(term)

assert len(lines) == 50
for term, count in bi_counts.most_common(MAX):
    lines.append(' '.join(term))

assert len(lines) == 100
with open(OUT_FILE, 'w') as out:
    out.write('\n'.join(lines))
!cat -n {OUT_FILE}

     1	the
     2	of
     3	and
     4	in
     5	a
     6	to
     7	is
     8	for
     9	with
    10	as
    11	that
    12	by
    13	are
    14	at
    15	be
    16	this
    17	on
    18	from
    19	can
    20	was
    21	an
    22	which
    23	it
    24	we
    25	energy
    26	fig
    27	have
    28	or
    29	s
    30	has
    31	high
    32	using
    33	these
    34	were
    35	used
    36	cells
    37	c
    38	m
    39	been
    40	e
    41	cell
    42	also
    43	d
    44	temperature
    45	two
    46	between
    47	not
    48	than
    49	i
    50	such
    51	of the
    52	in the
    53	to the
    54	on the
    55	and the
    56	can be
    57	for the
    58	from the
    59	that the
    60	with the
    61	at the
    62	is the
    63	in fig
    64	by the
    65	of a
    66	in a
    67	due to
    68	with a
    69	to be
    70	it is
    71	as a
    72	has been
    73	shown in
    74	as the
    75	in this
    76	such as
    77	et al
    78	to a
    79	between the
    80	is a
    81	have bee